# Dependencias

In [57]:
import numpy as np
import pandas as pd
import missingno as msgno

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf

from datetime import date,time,datetime
from dateutil.relativedelta import relativedelta as rd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.mixture import GaussianMixture
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.manifold import MDS,TSNE
from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering,KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score,mean_absolute_error
from sklearn.feature_selection import mutual_info_regression

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV

from varclushi import VarClusHi
from scipy import stats

from functools import reduce
from itertools import combinations

import json 
from glob import glob 
import os

from multiprocessing import Pool
from multiprocessing import get_context


from warnings import filterwarnings

pd.set_option('display.max_columns',None)
pd.set_option('display.float_format',lambda x:'%.2f'%x)
filterwarnings('ignore')
cf.go_offline()

# Cargamos los datos

In [58]:
path = "/Users/jesusarredondo/Documents/Diplomado Ciencia de Datos/Modulo5/Practica_Acciones/archive-2/full_history/*.csv"

In [59]:
"""Ordenamos cada una de las rutas de los archivos de la carpeta en orden alfabetico"""
larch = sorted(glob(path))
len(larch)

8387

In [60]:
"""Tomamos una muestra del 2-5% del total de nuestros archivos"""
larch= sorted(np.random.choice(larch,replace=False,size=int(len(larch)*0.05)))
len(larch)

419

In [61]:
8387*0.025

209.675

In [62]:
len(pd.read_csv(larch[0]))

7785

In [63]:
"""Tamaño total aproximado de registros"""
4680 * 8387

39251160

In [64]:
"""Tamaño muestra de registros"""
4680 * 419

1960920

In [65]:
"""Comprobamos que los archivos sean unicos"""
len(set(larch)),len(larch)

(419, 419)

In [66]:
"""Función para leer cargar varios csv en distintos archivos"""
def leer_archivo(arch:str)->pd.DataFrame:
    """Lee el archivo de acciones y cambia el tipo de dato de acuerdo con lo mínimo necesario.
    Agrega el nombre de la acción (símbolo)

    Args:
        arch (str): La ruta completa del archivo.

    Returns:
        pd.DataFrame: Conjunto de datos tratado.
    """
    var = ['date','volume','open','high','low','close'] #Columnas a percerbar
    df = pd.read_csv(arch,usecols=var,dtype=str) #Leemos el archivo con la accion
    accion = arch.split('/')[-1].split('.')[0] #obtenemos el nombre de la accion obteniendo el ultimo elemento despues del / y el primer elemento de este antes del primer .
    for v in var:
        if v=='date':
            df[v] = pd.to_datetime(df[v],format='%Y-%m-%d').dt.date
        else:
            df[v] = pd.to_numeric(df[v],errors='coerce')
    df.insert(0,'stock',accion)
    return df

In [67]:
"""Ejemplo obtencion del nombre de una accion"""
print(larch[0])
print(larch[0].split("/")[-1].split(".")[0])

/Users/jesusarredondo/Documents/Diplomado Ciencia de Datos/Modulo5/Practica_Acciones/archive-2/full_history/AAN.csv
AAN


In [68]:
"""Función para correr procesos de manera paralela"""
from multiprocessing import get_context
def make_pool(func, params, threads):
    """
    Ejecuta de forma paralela múltiples llamadas a una función
    :param func: function, objeto función a paralelizar
    :param params: iterable, parámetros de evaluación paralela
    :param threads: int, número de hilos de multiprocesamiento
    :return: list, resultado de la ejecución paralela agrupada en una lista
    """
    pool = get_context("fork").Pool(threads)
    data = pool.starmap(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]

In [69]:
"""Unir csv sin paralelismo"""
df = pd.DataFrame()

# 1. Carga de los datos.
for f in larch:
    df = pd.concat([df, leer_archivo(f)])
df.shape

(1199581, 7)

In [70]:
"""Unir con palelerismo"""
df = pd.concat(make_pool(leer_archivo,[(a,) for a in larch],40),ignore_index=True)
display(df.shape)
df.head()

(1199581, 7)

,stock,date,volume,open,high,low,close
0,AAN,2020-07-02,413000,46.80,47.42,44.54,44.73
1,AAN,2020-07-01,597900,45.17,46.57,44.36,45.29
2,AAN,2020-06-30,622200,45.92,46.50,45.05,45.40
3,AAN,2020-06-29,649200,45.24,46.83,44.07,46.50
4,AAN,2020-06-26,1885900,43.47,44.41,42.85,44.32


In [71]:
df.head()

,stock,date,volume,open,high,low,close
0,AAN,2020-07-02,413000,46.80,47.42,44.54,44.73
1,AAN,2020-07-01,597900,45.17,46.57,44.36,45.29
2,AAN,2020-06-30,622200,45.92,46.50,45.05,45.40
3,AAN,2020-06-29,649200,45.24,46.83,44.07,46.50
4,AAN,2020-06-26,1885900,43.47,44.41,42.85,44.32


# Ingeniería de Caracteristicas

In [72]:
"""Creamos la columna de diferencia que nos indica la distancia del precio alto y bajo de una accion"""
df['dif'] = df['high']-df['low']

In [73]:
"""Volvemos a cargar las variables continuas"""
varc = ['volume','open','high','low','close','dif']

In [74]:
"""Fecha maxima dentro del dataset"""
df["date"].max()

datetime.date(2020, 7, 2)

In [75]:
"""Cuantos registros hay por año"""
df["date"].map(lambda x:x.strftime ('%Y')).value_counts().sort_index()

1970      508
1971      506
1972      648
1973     1141
1974     1264
1975     1292
1976     1504
1977     1495
1978     1633
1979     1700
1980     4308
1981     5727
1982     6320
1983     6984
1984     7214
1985     7438
1986     7973
1987     8889
1988     8986
1989     9859
1990    10629
1991    11543
1992    12934
1993    14128
1994    14931
1995    15405
1996    17071
1997    19392
1998    20467
1999    21149
2000    23100
2001    23963
2002    25729
2003    27041
2004    28151
2005    30295
2006    33626
2007    37696
2008    40333
2009    41553
2010    44113
2011    46767
2012    48765
2013    52366
2014    56683
2015    61079
2016    65277
2017    70189
2018    75181
2019    81510
2020    43126
Name: date, dtype: int64

In [76]:
"""Nos quedamos con los ultimos 10 años"""
df = df.loc[df['date']>=pd.to_datetime('2011-01-01')].reset_index(drop=True)

In [77]:
df.shape

(600943, 8)

In [78]:
"""Detectamos las acciones que tengan cotizacion dos veces en un mismo dia"""
aux = df[["stock","date","volume"]].groupby(by=["stock","date"]).count().sort_values(by = "volume", ascending=False)
repetidos = list(set(dict(aux.loc[aux["volume"]>1].index).keys()))
repetidos

[]

In [79]:
"""Eliminamos las acciones detectadas"""
df = df.loc[~df['stock'].isin(repetidos)].reset_index(drop=True)
df.shape 

(600943, 8)

In [80]:
"""Comprobamos que ya no haya repetidos"""
df[["stock","date","volume"]].groupby(by=["stock","date"]).count().sort_values(by = "volume", ascending=False)
del(aux) #Eliminamos la variable aux para que no ocupe memoria

In [81]:
#Creamos el catalogo de fechas (que es un dataframe) por dia, eliminando duplicados
catfh = df[['date']].drop_duplicates().sort_values(by='date').reset_index(drop=True)

#Creamos la columna mes, donde en el catalogo de fechas les sacamos su año y su mes 
catfh['mes'] = catfh['date'].map(lambda x:x.strftime('%Y%m'))

#Creamos la variable auxiliar donde se elimina los meses duplicados, recordando que la columna mes tiene el año y el mes
aux = catfh[['mes']].drop_duplicates().sort_values(by='mes').reset_index(drop=True)

#A cada mes detectado y ordenado de menor a mayor, se agrega una columna de id_mes la cual esta dado por el indice del mes-año + 1
aux.insert(0,'id_mes',aux.index+1)

#Combinamos la tabla auxiliar y la de catalogo de fechas para tener un indicador del mes al que pertenece la transacción dado que la unidad muestral del catalogo
#de fechas es por dia
catfh = catfh.merge(aux,on='mes',how='inner')
catfh["id_mes"].max()
catfh

,date,mes,id_mes
0,2011-01-03,201101,1
1,2011-01-04,201101,1
2,2011-01-05,201101,1
3,2011-01-06,201101,1
4,2011-01-07,201101,1
...,...,...,...
2386,2020-06-26,202006,114
2387,2020-06-29,202006,114
2388,2020-06-30,202006,114
2389,2020-07-01,202007,115


In [82]:
"""Tomamos todas las fechas menores al último mes dado que no esta completo"""
catfh = catfh.loc[catfh['id_mes']<115].reset_index(drop=True)
catfh.tail()

,date,mes,id_mes
2384,2020-06-24,202006,114
2385,2020-06-25,202006,114
2386,2020-06-26,202006,114
2387,2020-06-29,202006,114
2388,2020-06-30,202006,114


In [83]:
catfh.filter(like='mes')

,mes,id_mes
0,201101,1
1,201101,1
2,201101,1
3,201101,1
4,201101,1
...,...,...
2384,202006,114
2385,202006,114
2386,202006,114
2387,202006,114


In [84]:
"""Eliminamos las columnas que tengan mes en su string"""
df.drop(df.filter(like='mes').columns,axis=1,inplace=True)
df

,stock,date,volume,open,high,low,close,dif
0,AAN,2020-07-02,413000,46.80,47.42,44.54,44.73,2.88
1,AAN,2020-07-01,597900,45.17,46.57,44.36,45.29,2.21
2,AAN,2020-06-30,622200,45.92,46.50,45.05,45.40,1.45
3,AAN,2020-06-29,649200,45.24,46.83,44.07,46.50,2.76
4,AAN,2020-06-26,1885900,43.47,44.41,42.85,44.32,1.56
...,...,...,...,...,...,...,...,...
600938,ZVO,2019-04-22,44500,5.98,6.22,5.87,5.92,0.36
600939,ZVO,2019-04-18,50800,5.93,6.11,5.89,6.01,0.22
600940,ZVO,2019-04-17,124300,6.13,6.24,5.84,5.93,0.40
600941,ZVO,2019-04-16,82900,6.38,6.38,6.15,6.15,0.23


In [85]:
"""Juntamos la tabla df y el catalogo de fechas"""
df = df.merge(catfh,on='date',how='inner')
df.head()

,stock,date,volume,open,high,low,close,dif,mes,id_mes
0,AAN,2020-06-30,622200,45.92,46.50,45.05,45.40,1.45,202006,114
1,ACI,2020-06-30,5047200,15.57,15.90,15.45,15.77,0.45,202006,114
2,ACM,2020-06-30,853500,37.47,37.99,37.00,37.58,0.99,202006,114
3,ADI,2020-06-30,2400500,119.52,123.20,119.01,122.64,4.19,202006,114
4,AFMC,2020-06-30,300,16.73,16.93,16.73,16.93,0.20,202006,114


In [86]:
# Son menos dado que ya no se tienen las fechas del ultimo mes
df.shape

(600327, 10)

In [87]:
df["date"].max()

datetime.date(2020, 6, 30)

In [88]:
"""Corroboramos que se hayan pasado todos los id_mes"""
df["id_mes"].unique()

array([114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102,
       101, 100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,
        88,  87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,
        75,  74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,
        62,  61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,
        49,  48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,
        36,  35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,
        23,  22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,
        10,   9,   8,   7,   6,   5,   4,   3,   2,   1])

In [89]:
"""Definimos las ventanas de tiempo"""
vobs = 12 #Ventana de observacion, cuantos meses se va a usar para predecir la siguiente
vdes = 1 #Ventana de desempeño
step = 3 #Sub ventana, es la que nos da el salto en el tiempo, a corto, mediano y largo plazo.

In [90]:
"""Funcion que crea la nuevas variables, en funcion a un ancla y ta ventena de tiempo"""
def ingX(df:pd.DataFrame,ancla:int,sub:int)->pd.DataFrame:
    """Crea la representación vectorial de la acción anclada a un momento particular del tiempo
    utilizando una sub-ventana de tiempo determinada.

    Args:
        df (pd.DataFrame): Datos Transaccionales
        ancla (int): ancla que fungirá como t0
        sub (int): sub-ventana de tiempo (en meses)

    Returns:
        pd.DataFrame: Matriz de predictoras para la combinación ancla-acción
    """
    #Creamos una variable auxiliar donde va a estar determinada por el ancla escogida y va a empeza en el ancla menos la sub ventana mas uno dado que empezamos a contar en 1
    aux = df.loc[(df['id_mes']<=ancla)&(df['id_mes']>=(ancla-sub+1))].copy().reset_index(drop=True)
    
    #Creamos una tabla pivote donde mostramos el precio de cierre de los stock que cerraron por dia
    piv = aux.pivot(index='stock',columns='date',values='close')

    #Cambiamos el nombre de la columna por el número de la observación
    piv.columns = range(len(piv.columns))

    #Sacamos la diferencia del periodo ti - ti-1, ademas identificamos si hay un incremento o decremento en el periodo siguiente
    for i,t in enumerate(zip(piv.columns[1:],piv.columns)):
        piv[f'dif_{i}'] = (piv[t[0]]-piv[t[1]])/piv[t[1]]
        piv[f'inc_{i}'] = (piv[t[1]]>piv[t[0]])
        piv[f'dec_{i}'] = (piv[t[1]]<piv[t[0]])
    piv[f'x_incpct_prom_{sub}'] = piv.filter(like='dif_').mean(axis=1) #sacamos el incremento primedio en la subventana de tiempo deseada
    piv.drop(piv.filter(like='dif_').columns,axis=1,inplace=True) #Eliminamos todas las columnas que tengan dif_
    
    #Usando la misma logica del inc promedio se calcula el promedio de precios con decremento e incremento
    for t in ['inc','dec']:
        piv[f'x_num_{t}precio_{sub}'] = piv.filter(like=f'{t}_').mean(axis=1)
        piv.drop(piv.filter(like=f'{t}_').columns,axis=1,inplace=True) #Eliminamos las observaciones de incremento y decremento por periodo de obs i
    
    piv = piv.filter(like='x_').reset_index() #Nos quedamos con las columnas que empiecen por x_
    
    #Creamos una columna donde introducimos el ancla de la cual estamos partiendo
    piv.insert(1,'ancla',ancla) 

    #Hacemos combinaciones de las variables continuas para obtener el ratio entre variables
    for v1,v2 in combinations(varc,2):
        aux[f'ratio_{v1}_{v2}'] = aux[v1]/aux[v2]

    #Ordenamos los ratios en orden ascendente
    ratios = sorted(aux.filter(like='ratio_').columns)

    #En el dataframe auxiliar hacemos funciones de agregacion sobre las variables continuas y los nuevos ratios calculados
    aux = aux[['stock']+varc+ratios].groupby('stock').agg(['min','max','mean','median','std'])

    #Renombramos las columnas del dataframe para que nos diga la subventana a la que pertenece
    aux.columns = [f'x_{f}_{v}_{sub}' for f,v in aux.columns]
    aux.reset_index(inplace=True)

    #Incertamos la colymna ancla en el df auxiliar
    aux.insert(1,'ancla',ancla)

    #Finalmente juntamos la tabla auxiliar con la tabla pivote para obtener los incrementos, decrementos y diferencias promedio
    return aux.merge(piv,on=['stock','ancla'],how='inner')

In [91]:
anclai, anclaf = df['id_mes'].min(),df['id_mes'].max()
anclai, anclaf = anclai+vobs-1,anclaf-vdes
anclai, anclaf

(12, 113)

In [92]:
ancla = 113
sub = 2
aux = df.loc[(df['id_mes']<=ancla)&(df['id_mes']>=(ancla-sub+1))].copy().reset_index(drop=True)
piv = aux.pivot(index='stock',columns='date',values='close')
piv

date,2020-04-01,2020-04-02,2020-04-03,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-26,2020-05-27,2020-05-28,2020-05-29
stock,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAN,19.74,19.17,18.62,22.14,22.88,23.67,25.24,24.69,25.96,23.25,23.01,24.59,24.71,23.26,23.62,24.14,25.70,26.82,27.93,32.33,31.91,30.51,30.96,29.40,28.80,33.35,35.78,34.51,32.90,29.69,32.08,31.30,33.37,32.27,33.76,33.70,33.98,36.31,38.38,36.24,36.91
ACI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5600.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACM,27.19,27.53,26.19,28.27,29.30,31.32,33.08,32.79,33.61,31.83,31.13,32.85,32.46,31.58,32.25,33.16,33.96,34.76,35.90,36.88,36.26,35.01,34.40,35.30,35.43,35.15,36.14,34.98,32.86,30.15,30.99,32.02,34.69,33.92,35.52,35.63,35.82,38.17,39.20,38.68,38.77
ADI,84.09,87.70,86.56,95.80,96.25,101.40,98.79,99.67,103.02,100.45,101.83,102.80,100.91,96.47,102.12,101.81,105.92,108.03,107.61,113.34,109.60,103.64,103.40,105.94,105.25,106.35,109.83,108.42,106.60,103.80,105.49,101.80,108.34,106.33,114.57,111.01,111.06,112.70,114.68,110.96,112.95
ADMA,2.46,2.59,3.09,3.18,2.94,2.77,2.73,2.73,2.84,2.75,2.70,2.69,2.80,2.67,2.67,2.83,3.06,3.18,2.93,3.09,2.95,3.01,3.14,3.17,3.20,2.62,2.41,2.67,2.55,2.49,2.49,2.62,2.60,2.62,2.76,2.91,2.84,2.94,3.03,2.86,3.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XSLV,30.74,30.88,29.54,31.61,31.98,33.37,35.77,34.21,34.55,32.75,32.10,33.60,32.61,32.27,32.08,32.35,32.45,33.82,34.98,36.83,35.14,33.77,33.32,33.19,32.37,32.73,33.98,32.87,31.31,29.97,30.29,30.37,32.20,31.51,32.29,32.26,32.31,33.48,34.78,33.91,33.55
YCS,73.55,74.64,75.29,76.42,75.82,75.92,75.27,74.26,73.53,74.25,74.37,74.23,74.39,74.45,74.42,74.30,73.97,73.78,73.25,72.89,73.88,73.20,73.02,72.72,72.16,72.44,73.01,74.26,73.67,73.42,73.80,73.72,73.85,74.33,74.11,74.14,74.17,74.04,74.37,74.22,74.43
ZKIN,0.84,0.83,0.83,0.84,0.86,0.83,1.00,1.17,1.10,1.11,1.09,1.05,1.10,1.10,1.09,1.17,1.21,1.19,1.18,1.19,1.16,1.12,1.12,1.11,1.14,1.13,1.14,1.44,1.54,1.31,1.28,1.18,1.19,1.21,1.25,1.25,1.21,1.16,1.14,1.12,1.08


In [93]:
piv.columns = range(len(piv.columns))
piv

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
stock,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAN,19.74,19.17,18.62,22.14,22.88,23.67,25.24,24.69,25.96,23.25,23.01,24.59,24.71,23.26,23.62,24.14,25.70,26.82,27.93,32.33,31.91,30.51,30.96,29.40,28.80,33.35,35.78,34.51,32.90,29.69,32.08,31.30,33.37,32.27,33.76,33.70,33.98,36.31,38.38,36.24,36.91
ACI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5600.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACM,27.19,27.53,26.19,28.27,29.30,31.32,33.08,32.79,33.61,31.83,31.13,32.85,32.46,31.58,32.25,33.16,33.96,34.76,35.90,36.88,36.26,35.01,34.40,35.30,35.43,35.15,36.14,34.98,32.86,30.15,30.99,32.02,34.69,33.92,35.52,35.63,35.82,38.17,39.20,38.68,38.77
ADI,84.09,87.70,86.56,95.80,96.25,101.40,98.79,99.67,103.02,100.45,101.83,102.80,100.91,96.47,102.12,101.81,105.92,108.03,107.61,113.34,109.60,103.64,103.40,105.94,105.25,106.35,109.83,108.42,106.60,103.80,105.49,101.80,108.34,106.33,114.57,111.01,111.06,112.70,114.68,110.96,112.95
ADMA,2.46,2.59,3.09,3.18,2.94,2.77,2.73,2.73,2.84,2.75,2.70,2.69,2.80,2.67,2.67,2.83,3.06,3.18,2.93,3.09,2.95,3.01,3.14,3.17,3.20,2.62,2.41,2.67,2.55,2.49,2.49,2.62,2.60,2.62,2.76,2.91,2.84,2.94,3.03,2.86,3.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XSLV,30.74,30.88,29.54,31.61,31.98,33.37,35.77,34.21,34.55,32.75,32.10,33.60,32.61,32.27,32.08,32.35,32.45,33.82,34.98,36.83,35.14,33.77,33.32,33.19,32.37,32.73,33.98,32.87,31.31,29.97,30.29,30.37,32.20,31.51,32.29,32.26,32.31,33.48,34.78,33.91,33.55
YCS,73.55,74.64,75.29,76.42,75.82,75.92,75.27,74.26,73.53,74.25,74.37,74.23,74.39,74.45,74.42,74.30,73.97,73.78,73.25,72.89,73.88,73.20,73.02,72.72,72.16,72.44,73.01,74.26,73.67,73.42,73.80,73.72,73.85,74.33,74.11,74.14,74.17,74.04,74.37,74.22,74.43
ZKIN,0.84,0.83,0.83,0.84,0.86,0.83,1.00,1.17,1.10,1.11,1.09,1.05,1.10,1.10,1.09,1.17,1.21,1.19,1.18,1.19,1.16,1.12,1.12,1.11,1.14,1.13,1.14,1.44,1.54,1.31,1.28,1.18,1.19,1.21,1.25,1.25,1.21,1.16,1.14,1.12,1.08


In [94]:
(piv[0]>piv[1])

stock
AAN      True
ACI     False
ACM     False
ADI     False
ADMA    False
        ...  
XSLV    False
YCS     False
ZKIN     True
ZROZ    False
ZVO      True
Length: 361, dtype: bool

In [95]:
for i,t in enumerate(zip(piv.columns[1:],piv.columns)):
    print(i,t)
    print(t[0],t[1])

0 (1, 0)
1 0
1 (2, 1)
2 1
2 (3, 2)
3 2
3 (4, 3)
4 3
4 (5, 4)
5 4
5 (6, 5)
6 5
6 (7, 6)
7 6
7 (8, 7)
8 7
8 (9, 8)
9 8
9 (10, 9)
10 9
10 (11, 10)
11 10
11 (12, 11)
12 11
12 (13, 12)
13 12
13 (14, 13)
14 13
14 (15, 14)
15 14
15 (16, 15)
16 15
16 (17, 16)
17 16
17 (18, 17)
18 17
18 (19, 18)
19 18
19 (20, 19)
20 19
20 (21, 20)
21 20
21 (22, 21)
22 21
22 (23, 22)
23 22
23 (24, 23)
24 23
24 (25, 24)
25 24
25 (26, 25)
26 25
26 (27, 26)
27 26
27 (28, 27)
28 27
28 (29, 28)
29 28
29 (30, 29)
30 29
30 (31, 30)
31 30
31 (32, 31)
32 31
32 (33, 32)
33 32
33 (34, 33)
34 33
34 (35, 34)
35 34
35 (36, 35)
36 35
36 (37, 36)
37 36
37 (38, 37)
38 37
38 (39, 38)
39 38
39 (40, 39)
40 39


In [96]:
piv[t[1]]>piv[t[0]]

stock
AAN     False
ACI     False
ACM     False
ADI     False
ADMA    False
        ...  
XSLV     True
YCS     False
ZKIN     True
ZROZ    False
ZVO     False
Length: 361, dtype: bool

In [97]:
for v1,v2 in combinations(varc,2):
    print(v1,v2)

volume open
volume high
volume low
volume close
volume dif
open high
open low
open close
open dif
high low
high close
high dif
low close
low dif
close dif


In [98]:
varc

['volume', 'open', 'high', 'low', 'close', 'dif']

In [99]:
"""Nombramos a las anclas"""
anclai, anclaf = df['id_mes'].min(),df['id_mes'].max()
anclai, anclaf = anclai+vobs-1,anclaf-vdes
anclai, anclaf

(12, 113)

In [100]:
"""Nombramos a nuestra unidad muestral"""
um = ['stock','ancla']

In [101]:
"""Funciones de cruce"""
cruzar = lambda x,y: pd.merge(x,y,on=um,how='outer') #Unimos el df x con y, quedandonos con las claves que esten x
apilar = lambda x,y: pd.concat([x,y],ignore_index=True) #Apilamos las filas de dos dataframe

In [102]:
def repVect (ancla:int)->pd.DataFrame:
    """Genera la represntación vectorial para las sub-anclas

    Args:
        ancla (int): ancla específica

    Returns:
        pd.DataFrame: Matriz de predictoras
    """
    return reduce(cruzar,map(lambda sub:ingX(df,ancla,sub),range(step,vobs+step,step)))

In [103]:
X = reduce(apilar,make_pool(repVect,[(a,) for a in range(anclai,anclaf+1)],20))
X

,stock,ancla,x_volume_min_3,x_volume_max_3,x_volume_mean_3,x_volume_median_3,x_volume_std_3,x_open_min_3,x_open_max_3,x_open_mean_3,x_open_median_3,x_open_std_3,x_high_min_3,x_high_max_3,x_high_mean_3,x_high_median_3,x_high_std_3,x_low_min_3,x_low_max_3,x_low_mean_3,x_low_median_3,x_low_std_3,x_close_min_3,x_close_max_3,x_close_mean_3,x_close_median_3,x_close_std_3,x_dif_min_3,x_dif_max_3,x_dif_mean_3,x_dif_median_3,x_dif_std_3,x_ratio_close_dif_min_3,x_ratio_close_dif_max_3,x_ratio_close_dif_mean_3,x_ratio_close_dif_median_3,x_ratio_close_dif_std_3,x_ratio_high_close_min_3,x_ratio_high_close_max_3,x_ratio_high_close_mean_3,x_ratio_high_close_median_3,x_ratio_high_close_std_3,x_ratio_high_dif_min_3,x_ratio_high_dif_max_3,x_ratio_high_dif_mean_3,x_ratio_high_dif_median_3,x_ratio_high_dif_std_3,x_ratio_high_low_min_3,x_ratio_high_low_max_3,x_ratio_high_low_mean_3,x_ratio_high_low_median_3,x_ratio_high_low_std_3,x_ratio_low_close_min_3,x_ratio_low_close_max_3,x_ratio_low_close_mean_3,x_ratio_low_close_median_3,x_ratio_low_close_std_3,x_ratio_low_dif_min_3,x_ratio_low_dif_max_3,x_ratio_low_dif_mean_3,x_ratio_low_dif_median_3,x_ratio_low_dif_std_3,x_ratio_open_close_min_3,x_ratio_open_close_max_3,x_ratio_open_close_mean_3,x_ratio_open_close_median_3,x_ratio_open_close_std_3,x_ratio_open_dif_min_3,x_ratio_open_dif_max_3,x_ratio_open_dif_mean_3,x_ratio_open_dif_median_3,x_ratio_open_dif_std_3,x_ratio_open_high_min_3,x_ratio_open_high_max_3,x_ratio_open_high_mean_3,x_ratio_open_high_median_3,x_ratio_open_high_std_3,x_ratio_open_low_min_3,x_ratio_open_low_max_3,x_ratio_open_low_mean_3,x_ratio_open_low_median_3,x_ratio_open_low_std_3,x_ratio_volume_close_min_3,x_ratio_volume_close_max_3,x_ratio_volume_close_mean_3,x_ratio_volume_close_median_3,x_ratio_volume_close_std_3,x_ratio_volume_dif_min_3,x_ratio_volume_dif_max_3,x_ratio_volume_dif_mean_3,x_ratio_volume_dif_median_3,x_ratio_volume_dif_std_3,x_ratio_volume_high_min_3,x_ratio_volume_high_max_3,x_ratio_volume_high_mean_3,x_ratio_volume_high_median_3,x_ratio_volume_high_std_3,x_ratio_volume_low_min_3,x_ratio_volume_low_max_3,x_ratio_volume_low_mean_3,x_ratio_volume_low_median_3,x_ratio_volume_low_std_3,x_ratio_volume_open_min_3,x_ratio_volume_open_max_3,x_ratio_volume_open_mean_3,x_ratio_volume_open_median_3,x_ratio_volume_open_std_3,x_incpct_prom_3,x_num_incprecio_3,x_num_decprecio_3,x_volume_min_6,x_volume_max_6,x_volume_mean_6,x_volume_median_6,x_volume_std_6,x_open_min_6,x_open_max_6,x_open_mean_6,x_open_median_6,x_open_std_6,x_high_min_6,x_high_max_6,x_high_mean_6,x_high_median_6,x_high_std_6,x_low_min_6,x_low_max_6,x_low_mean_6,x_low_median_6,x_low_std_6,x_close_min_6,x_close_max_6,x_close_mean_6,x_close_median_6,x_close_std_6,x_dif_min_6,x_dif_max_6,x_dif_mean_6,x_dif_median_6,x_dif_std_6,x_ratio_close_dif_min_6,x_ratio_close_dif_max_6,x_ratio_close_dif_mean_6,x_ratio_close_dif_median_6,x_ratio_close_dif_std_6,x_ratio_high_close_min_6,x_ratio_high_close_max_6,x_ratio_high_close_mean_6,x_ratio_high_close_median_6,x_ratio_high_close_std_6,x_ratio_high_dif_min_6,x_ratio_high_dif_max_6,x_ratio_high_dif_mean_6,x_ratio_high_dif_median_6,x_ratio_high_dif_std_6,x_ratio_high_low_min_6,x_ratio_high_low_max_6,x_ratio_high_low_mean_6,x_ratio_high_low_median_6,x_ratio_high_low_std_6,x_ratio_low_close_min_6,x_ratio_low_close_max_6,x_ratio_low_close_mean_6,x_ratio_low_close_median_6,x_ratio_low_close_std_6,x_ratio_low_dif_min_6,x_ratio_low_dif_max_6,x_ratio_low_dif_mean_6,x_ratio_low_dif_median_6,x_ratio_low_dif_std_6,x_ratio_open_close_min_6,x_ratio_open_close_max_6,x_ratio_open_close_mean_6,x_ratio_open_close_median_6,x_ratio_open_close_std_6,x_ratio_open_dif_min_6,x_ratio_open_dif_max_6,x_ratio_open_dif_mean_6,x_ratio_open_dif_median_6,x_ratio_open_dif_std_6,x_ratio_open_high_min_6,x_ratio_open_high_max_6,x_ratio_open_high_mean_6,x_ratio_open_high_median_6,x_ratio_open_high_std_6,x_ratio_open_low_min_6,x_ratio_open_low_max_6,x_ratio_open_low_mean_6,x_ratio_open_low_median_6,x_ratio_open_lo

In [104]:
def ingY(df:pd.DataFrame,ancla:int)->pd.DataFrame:
    """Crea la variable objetivo para un ancla específica

    Args:
        df (pd.DataFrame): Datos Transaccionales
        ancla (int): ancla que fungirá como t0

    Returns:
        pd.DataFrame: Vector respuesta para la combinación ancla-acción
    """
    aux = df.loc[(df['id_mes']>ancla)&(df['id_mes']<=(ancla+vdes))].copy().reset_index(drop=True)[['stock','date','close','id_mes']]
    aux.sort_values(by=['stock','date','id_mes'],inplace=True)
    aux = aux.groupby(['stock','id_mes']).last().reset_index()
    aux.insert(1,'ancla',ancla)
    aux.rename(columns={'close':'cierre_sig_mes'},inplace=True)
    return aux.drop(['date','id_mes'],axis=1)

In [105]:
y = reduce(apilar,map(lambda ancla:ingY(df,ancla),range(anclai,anclaf+1)))
y.shape

(27256, 3)

In [106]:
X.shape,y.shape

((27128, 434), (27256, 3))

In [107]:
y

,stock,ancla,cierre_sig_mes
0,AAN,12,26.61
1,ACI,12,69.15
2,ACM,12,22.89
3,ADI,12,39.13
4,AGQ,12,236.32
...,...,...,...
27251,YCS,113,74.58
27252,ZGYHR,113,0.20
27253,ZKIN,113,1.41
27254,ZROZ,113,172.92


In [108]:
tad = X.merge(y,on=um,how='inner')
tad.to_pickle('tad_acciones.pkl')